In [36]:
import json
import requests
import time
from data import *
import pickle

In [43]:
eiggRawData = eiggRawData()
eiggRawData = eiggRawData["Scientific name"].str.lower()
eiggRawData = set(eiggRawData)
len(eiggRawData)

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2460

In [65]:
def preparedName(name,limit):
    name = name.split(" ")[:limit]
    return "+".join(name)

def validateSingleName(name,limit=100):
    callToValidateName = requests.get('http://resolver.globalnames.org/name_resolvers.json?names='+preparedName(name,limit))
    jsonRes = callToValidateName.json()['data'][0]
    #time.sleep(2)    =>  Politeness for server
    try:
        if not jsonRes['is_known_name']:
            return jsonRes['results']['canonical_form']
        else:
            return " ".join(name.split(" ")[:limit])
    except:
        print("error occured at "+name)
        return name+" error_loc"

def validateListedEiggSpecies(speciesList):
    for species in eiggRawData:
        indivResult = validateSingleName(species)
        validatedSpeciesResultDict[species] = indivResult
    
    print(len(validatedSpeciesResultDict))
    with open("PartialEiggVerified",'wb') as f:
        pickle.dump(validatedSpeciesResultDict,f)
        
    failedComputations = list(filter(lambda x: "error_loc" in validatedSpeciesResultDict[x],validatedSpeciesResultDict.keys()))
    return failedComputations

In [45]:
validateListedEiggSpecies(eiggRawData)

# How do we deal with these 113 failed entries?

Doing it manually feels wrong...Google searching 113 records doesn't seem appealing

We can notice; the first two keywords of the phrase indicate the species. Anything after that, which is what destroys the APIs ability to tell if it's correct, is metadata.

Hence let's refire the requests, but only on the first two keywords

In [47]:
failedComputationDict = dict(zip(failedComputations, failedComputations))
len(failedComputationDict)

113

In [67]:
amendedDict = {}
failed = []
for failedName in failedComputationDict:
    validatedName = validateSingleName(failedName,2)
    if "error_loc" in validatedName:
        failed.append(failedName)
    else:
        amendedDict[failedName] = validatedName

error occured at trochulus (trochulus) hispidus
error occured at porifera indet crusts
error occured at cochlicopa cf. lubricella
error occured at euconulus agg.
error occured at anisus (anisus) leucostoma
error occured at diatoms colonial
error occured at rhodophycota indet. (non-calc. crusts)
error occured at taraxacum agg.
error occured at aphodius fimetarius/pedellus agg.
error occured at lichenicolous fungus (unidentified)
error occured at cochlicopa cf. lubrica
error occured at trochulus (trochulus) striolatus
error occured at maerl indet
error occured at bryozoa indet crusts
error occured at euconulus (euconulus) cf. fulvus
error occured at lichen (unidentified)
error occured at notonecta (notonecta) obliqua
error occured at filamentous brown algae
error occured at leiostyla (leiostyla) anglica
error occured at rubus sect. corylifolii
error occured at anthus spinoletta/petrosus agg.
error occured at lithobius (lithobius) forficatus
error occured at gyraulus (armiger) crista
erro

In [68]:
len(amendedDict)
print(amendedDict['carex hostiana x viridula = c. x fulva'])

carex hostiana


# Result

We saved 81 of the 113 errored plants. The tradeoff was technically some useful information- the metadata we saw, such as "x fragilis"- a specific type of the species we are studying.

However, we will proceed with the assumption that animals don't particularly care what type of species something is, if it will be their dinner. E.g. a grasshopper won't particularly care what specific type of wheat it eats; as long as it's the same species, it will be valid in the food chain.

## I will fix the rest manually

In [69]:
failed

['trochulus (trochulus) hispidus',
 'porifera indet crusts',
 'cochlicopa cf. lubricella',
 'euconulus agg.',
 'anisus (anisus) leucostoma',
 'diatoms colonial',
 'rhodophycota indet. (non-calc. crusts)',
 'taraxacum agg.',
 'aphodius fimetarius/pedellus agg.',
 'lichenicolous fungus (unidentified)',
 'cochlicopa cf. lubrica',
 'trochulus (trochulus) striolatus',
 'maerl indet',
 'bryozoa indet crusts',
 'euconulus (euconulus) cf. fulvus',
 'lichen (unidentified)',
 'notonecta (notonecta) obliqua',
 'filamentous brown algae',
 'leiostyla (leiostyla) anglica',
 'rubus sect. corylifolii',
 'anthus spinoletta/petrosus agg.',
 'lithobius (lithobius) forficatus',
 'gyraulus (armiger) crista',
 'cochlicella (cochlicella) acuta',
 'haliclona (rhizoniera) viscosa',
 'columella agg.',
 'diatoms film',
 'encrusting algae indet.',
 'zostera (zostera) marina',
 'porania (porania) pulvillus',
 'platyhypnidium ripariodes',
 'vertebrata fucoides']

In [70]:
amendedDict['trochulus (trochulus) hispidus'] = 'trochulus hispidus'
amendedDict['porifera indet crusts'] = 'porifera indet crusts' #essentially a sea sponge- not recognized by resolver
amendedDict['cochlicopa cf. lubricella'] = 'cochlicopa lubricella'
amendedDict['anisus (anisus) leucostoma'] = 'anisus leucostoma'
amendedDict['rhodophycota indet. (non-calc. crusts)'] = 'rhodophycota'
amendedDict['taraxacum agg.'] = 'taraxacum officinale'
amendedDict['aphodius fimetarius/pedellus agg.'] = 'aphodius fimetarius'
amendedDict['cochlicopa cf. lubrica'] = 'cochlicopa lubrica'
amendedDict['trochulus (trochulus) striolatus'] = 'trochulus striolatus'
amendedDict['maerl indet'] = 'maerl'
amendedDict['bryozoa indet crusts'] = 'bryozoa'
amendedDict['euconulus (euconulus) cf. fulvus'] = 'euconulus fulvus'
amendedDict['notonecta (notonecta) obliqua'] = 'notonecta obliqua'
amendedDict['leiostyla (leiostyla) anglica'] = 'leiostyla anglica'
amendedDict['anthus spinoletta/petrosus agg.'] = 'anthus spinoletta'
amendedDict['lithobius (lithobius) forficatus'] = 'lithobius forficatus'
amendedDict['gyraulus (armiger) crista'] = 'gyraulus crista'
amendedDict['cochlicella (cochlicella) acuta'] = 'cochlicella acuta'
amendedDict['haliclona (rhizoniera) viscosa'] = 'haliclona viscosa'
amendedDict['columella agg.'] = 'columella'
amendedDict['zostera (zostera) marina'] = 'zostera marina'
amendedDict['porania (porania) pulvillus'] = 'porania pulvillus'
amendedDict['platyhypnidium ripariodes'] = 'platyhypnidium ripariodes'
amendedDict['vertebrata fucoides'] = 'vertebrata fucoides'

In [71]:
len(amendedDict)

105

In [75]:
fullSpeciesOfEiggDict = {}

with open("PartialEiggVerified",'rb') as f:
    partialDict = pickle.load(f)

for item in partialDict:
    if "error_loc" not in partialDict[item]:
        fullSpeciesOfEiggDict[item] = partialDict[item]
    
for item in amendedDict:
    fullSpeciesOfEiggDict[item] = amendedDict[item]
    
with open("EiggVerifiedSpeciesList",'wb') as f:
    pickle.dump(fullSpeciesOfEiggDict,f)


In [76]:
fullSpeciesOfEiggDict['carex hostiana x viridula = c. x fulva']

'carex hostiana'

In [77]:
len(fullSpeciesOfEiggDict)

2452

In [78]:
with open("EiggVerifiedSpeciesList",'rb') as f:
    testingPickleWrite = pickle.load(f)
    
print(len(testingPickleWrite))

2452
